In [1]:
import pandas as pd

In [2]:
df_clean = pd.read_excel('../df_clean.xlsx',index_col=0,sheet_name='Sheet1')

In [4]:
df_clean[["SPY","VBMFX"]]

,SPY,VBMFX
1993-04-30,-0.025588,0.007439
1993-05-31,0.026970,0.001403
1993-06-30,0.003607,0.018065
1993-07-31,-0.004854,0.006092
1993-08-31,0.038327,0.016933
...,...,...
2025-05-31,0.062845,-0.007126
2025-06-30,0.051386,0.015713
2025-07-31,0.023032,-0.002986
2025-08-31,0.020520,0.011542
